In [55]:
import tweepy
import csv
import pandas as pd
import numpy as np
import networkx as nx
import os
from Events_NER.TweetSegmenter import SEDTWikSegmenter

In [3]:
wiki_titles_file = "Events_NER/data/final.txt"
segmenter = SEDTWikSegmenter(wiki_titles_file, 4, 3, False)

Initializing SEDTWik Segmenter
SEDTWik Segmenter Ready



In [67]:
class Tweet():
    
    def __init__(self, status_response):
        self.id = status_response.id
        self._get_text(status_response)
        self.user_info = status_response.user
        self.entitites = status_response.entities
        self.json = status_response._json
        self.json['text'] = self.text
        arr = []
        for users in self.json['entities']['user_mentions']: 
            arr += [users['name']]
        self.json['entities']['user_mentions'] = arr
        arr = []
        for users in self.json['entities']['hashtags']: 
            arr += [users['text']]
        self.json['entities']['hashtags'] = arr
        
    def _get_text(self, status):
        if hasattr(status, "retweeted_status"):  # Check if Retweet
            try:
                self.text = status.retweeted_status.extended_tweet["full_text"]
            except AttributeError:
                self.text = status.retweeted_status.full_text
        else:
            try:
                self.text = status.extended_tweet["full_text"]
            except AttributeError:
                self.text = status.full_text
    
    #code for generating named entities of each tweet
    def _get_named_entities(self):
        return segmenter.tweet_segmentation(self.json)
        
    #code for generating event phrases
    def _get_event_phrases(self):
        return []
    
    def get_graph_entities(self):
        return self._get_named_entities() + self._get_event_phrases()
    
    def __hash__(self):
        return self.id
    def __eq__(self, other):
        return self.id == other.id


class TweetRetriever():

    def __init__(self):
        consumer_key = '95cMtk1vJvEEW2rlMR0kIU9lE'
        consumer_secret = 'pMQFi7LBdcudKDNZOokUJGS8mDxQanUv8spxBDdTLiwSZBuUOM'
        access_token = '1036313393114767360-BZ8Qpi02ghRvehhcITEIyl7SmGWmU6'
        access_token_secret = 'C7VAqGDhTdB424iBtEwF1CJI9YPTcvNvLjFmaCXENNv3G'
        auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
        #auth.set_access_token(access_token, access_token_secret)
        self.api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify = True)
    
    def _filterDuplicates(self, tweets):
        tweet_text = set()
        filtered_tweets = []
        for tweet in tweets:
            if tweet.text not in tweet_text:
                filtered_tweets += [tweet]
                tweet_text.add(tweet.text)
        return filtered_tweets

    def getTweets(self, hashtag, count = 10):
        tweets = []
        for status in tweepy.Cursor(self.api.search, q = hashtag + " -filter:retweets", count = count, tweet_mode = 'extended',
                                    lang = 'en',).items():
            tweets.append(Tweet(status))
        return self._filterDuplicates(tweets)

    

In [51]:
class GraphNode():
    
    def __init__(self, name):
        self.name = name
        self.tweets = set()
        self.value = 0
        
    def add_tweet(self, tweet):
        self.tweets.add(tweet)
    
    def common_tweets(self, other):
        return len(self.tweets.intersection(other.tweets))
    
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        return self.name == other.name
#     def __print__(self):
#         print(self.name)
    
class TweetGraph():
    
    def __init__(self, topic):
        self.topic = topic
        self.nodes = {}
        self.edge_map = {}
        
    def add_entity(self, name, tweet_ref):
        if name not in self.nodes:
            self.nodes[name] = GraphNode(name)
        self.nodes[name].add_tweet(tweet_ref)
    
    def add_edge(self, node1, node2):
        assert node1.name in self.nodes
        assert node2.name in self.nodes
        weight = node1.common_tweets(node2)
        self.edge_map.setdefault(node1.name, {}).setdefault(node2.name, weight)
        self.edge_map.setdefault(node2.name, {}).setdefault(node1.name, weight)
    
    def compute_all_edges(self):
        for node1 in self.nodes.values():
            for node2 in self.nodes.values():
                self.add_edge(node1, node2)
    
    def _get_pagerank_matrix(self):
        x = [[0 for _ in range(len(self.nodes))] for _ in range(len(self.nodes))]
        for i, node1 in enumerate(self.nodes.values()):
            wsum = 0
            for node2 in self.nodes.values():
                wsum += self.edge_map.get(node1.name, {}).get(node2.name, 0)
            for j, node2 in enumerate(self.nodes.values()):
                x[i][j] = self.edge_map.get(node1.name, {}).get(node2.name, 0)/wsum
        return np.array(x)
    
    def set_textrank_values(self, d = 0.85):
        rank_graph = nx.from_numpy_array(self._get_pagerank_matrix())
        node_scores = nx.pagerank(rank_graph, alpha = d)
        for i, node in enumerate(self.nodes.values()):
            node.value = node_scores[i]
    
    def get_weight(self, node1, node2):
        return self.edge_map.get(node1.name, {}).get(node2.name, 0)
    
    def get_topic_similarity(self, node):
        if node.name in self.topic:
            return len(node.tweets)
        return 1
    
    def get_all_node_values(self):
        arr = []
        for node in self.nodes.values():
            arr.append((node.name, node.value))
        return sorted(arr, key = lambda x: x[1])
    
    def get_avg_thres(self):
        values = self.get_all_node_values()
        return sum([i[1] for i in values])/len(values)
        
    def get_nodes_above_thres(self, thres = 1):
        nodes = []
        value_sum = 0
        for node in self.nodes.values():
            if node.value > thres:
                nodes.append(node)
                value_sum += node.value
        return nodes, value_sum
        

In [14]:
def createGraph(topic, tweets):
    tweetGraph = TweetGraph(topic)
    for tweet in tweets:
        graph_entities = tweet.get_graph_entities()
        for name in graph_entities:
            tweetGraph.add_entity(name, tweet)
    tweetGraph.compute_all_edges()
    tweetGraph.set_textrank_values()
    return tweetGraph

def partitionGraph(tweetGraph, alpha, beta, high_rank_thres = 1):
    
    #initialize highly ranked nodes and their total values sum
    high_ranked_nodes, total_value_sum = tweetGraph.get_nodes_above_thres(high_rank_thres)
    high_ranked_nodes = sorted(high_ranked_nodes, key = lambda x: x.value)
    partitions = []
    
    #partitioning loop
    while len(high_ranked_nodes):
        #entity set is the nodes in the partition
        entity_set = set()
        repr_node = high_ranked_nodes.pop()
        entity_set.add(repr_node)
        repr_node_topic_similarity = tweetGraph.get_topic_similarity(repr_node)
        value_sum = repr_node.value
        
        for node in high_ranked_nodes:
            node_edge_weight = tweetGraph.get_weight(repr_node, node)
            node_topic_similarity = tweetGraph.get_topic_similarity(node)
            
            if node_edge_weight/repr_node_topic_similarity > alpha and \
                node_topic_similarity/repr_node_topic_similarity > alpha:
                entity_set.add(node)
                value_sum += node.value
        
        if value_sum/total_value_sum > beta:
            temp = []
            for node in high_ranked_nodes:
                if node not in entity_set:
                    temp.append(node)
            high_ranked_nodes = temp
            
            partitions.append([])
            for node in list(entity_set):
                partitions[-1].append(node)
            
    return partitions
    

In [70]:
ret = TweetRetriever()

In [80]:
tweets = ret.getTweets('#x2', 1)

In [82]:
len(tweets)

28

In [192]:
tweets[0].get_graph_entities()

['do you', 'think', 'day6', 'superpower', 'day', 'day jae', 'mydays', 'sungjin', 'wonpil', 'dowoon', 'young k', 'jae', 'gravity', 'entropy', 'the book of us', 'day gravity', 'dayworldtour', 'day', '데이식스', 'sweet chaos', 'day sweet chaos']


['do you',
 'think',
 'day6',
 'superpower',
 'day',
 'day jae',
 'mydays',
 'sungjin',
 'wonpil',
 'dowoon',
 'young k',
 'jae',
 'gravity',
 'entropy',
 'the book of us',
 'day gravity',
 'dayworldtour',
 'day',
 '데이식스',
 'sweet chaos',
 'day sweet chaos']

In [193]:
len(tweets)

9

In [183]:
graph = createGraph(['gravity'], tweets)

['interstellar', 'space travel', 'time travel', 'tech', 'astronomy', 'astrophysics', 'space', 'science', 'physics']
['interstellar']
['introducing', 'next-gen', 'trickshot', 'solar', 'interstellar', 'out of this world']
['soundtrack', 'amazing', 'listen', 'do you', 'listen', 'interstellar', 'writers', 'writing', 'writing community']
['major major', 'thanks to', 'dropping', 'alison wonderland', 'sxmelectro', 'in rotation', 'insomniac records', 'interstellar', 'radio wonderland']
['rocket', 'science.', 'dog', 'science', 'physics', 'biology', 'cosmos', 'space', 'chemistry', 'blackhomeschoolers', 'astronomy', 'nasa', 'universe', 'spaceexploration', 'nature', 'interstellar', 'mars', 'solarsystem', 'art', 'study', 'science', 'school']
['hours', 'to go', 'neck', 'going to', 'don’t', 'let it happen', 'folks', 'votes', 'help', 'rob', 'ross', 'gotyoucovered', 'bond', 'interstellar', 'movie', 'moviechat', 'filmtwitter', 'movie poster', 'cinema']
['bunch', 'dancing', 'people', 'tight', 'clothes', 

In [184]:
graph.get_all_node_values()

[('space travel', 0.007132641058254577),
 ('time travel', 0.007132641058254577),
 ('tech', 0.007132641058254577),
 ('astrophysics', 0.007735622579121715),
 ('for the first time', 0.00866752878287172),
 ('comets', 0.008738474102461341),
 ('sky', 0.008738474102461341),
 ('telescope', 0.008738474102461341),
 ('maria lorca', 0.00898215702361635),
 ('the avengers', 0.00898215702361635),
 ('groundbreaking', 0.008982157023616351),
 ('sciencefiction', 0.008982157023616351),
 ('borisov', 0.00905087105960878),
 ('solar system', 0.00905087105960878),
 ('introducing', 0.009078096450958245),
 ('next-gen', 0.009078096450958245),
 ('trickshot', 0.009078096450958245),
 ('solar', 0.009078096450958245),
 ('out of this world', 0.009078096450958245),
 ('police unit', 0.009083684364101941),
 ('at the time', 0.009083684364101941),
 ('hard beat', 0.009083684364101941),
 ('trusty', 0.009083684364101943),
 ('the only', 0.009083684364101943),
 ('alistair mcknight', 0.009083684364101943),
 ('ghettos', 0.00908368

In [185]:
avg = graph.get_avg_thres()

In [186]:
partitions = partitionGraph(graph, 0.05, 0.01, high_rank_thres=avg)

In [187]:
def summarization(partitions, tweet_cutoff = 1):
    summary = []
    for part in partitions:
        tweet_set = set()
        for node in part:
            tweet_set = tweet_set.union(node.tweets)
        node_entity_count = []
        tweet_set = list(tweet_set)
        #print(tweet_set)
        for i, tweet in enumerate(tweet_set):
            count = 0
            for node in part:
                if node.name in tweet.text:
                    count += 1
            node_entity_count += [(i, count)]
        node_entity_count = sorted(node_entity_count, key = lambda x: x[1], reverse = True)
        #print(node_entity_count)
        for i in range(tweet_cutoff):
            summary += [tweet_set[node_entity_count[i][0]].text]
    return summary

In [188]:
for p in partitions:
    for p2 in p:
        print(p2.name, len(p2.tweets))
    print("====\n")

astronomy 2
space 3
science 3
interstellar 13
comet 2
physics 2
====



In [149]:
s = summarization(partitions)
print(s[0])
print("=====")
print(s[1])
print("=====")
print(s[2])


Renting is a far better and wiser option to choose when compared to buying furniture for your start up office. #entrepreneur #startupindia #startups #india #startupslist https://t.co/cAAXtLeM3F
=====
Within a span of less than a week, #OPEC held high-level meetings with the two fastest-growing oil consuming nations in the world – #India and #China – to further strengthen cooperation in support of oil market stability and the world economy. https://t.co/kNajp5ildg
=====
Funeral of soldier Zahid Farooqi, martyred yesterday, in his native Kalsan village of #Haveli district with full military honours.
In #IHK, #Kashmiris fight against #India &amp; in #AJK they fight for #Pakistan. That's what many #Indians either fail to understand or tend to ignore. https://t.co/qzM615KshC


In [160]:
list(partitions[0][0].tweets)[0].json['text']

'We need not to visualize a black hole now as the image is available. Powehi, a  black hole whose image was issued b… https://t.co/06Y5pQTmmU'

In [83]:
hashtags = "#x1, #mamavote, #welcomex1tothailand, #got7, #just_dohyon_day, #springday1000days, #exo, #darkbluekissep5, #bts, #peing, #nct127, #igot7, #nct, #ab6ix, #uwmaep1, #happy_dohyon_day, #straykids, #bigil, #monsta_x, #choseungyoun, #got7_callmyname, #kampsingapore2019, #fgo, #rt, #bolivia, #ps4share, #ssmbreignbeginssoon, #ateez, #thefacementhailand3, #got7now, #doyoung, #ayodhyaverdict, #sowetoderby, #sarileruneekevvaru, #astronaut, #johnny, #case1485, #sex, #yuta, #obsession, #kimwooseok, #kpopfestainbkk, #txt, #newprofilepic, #ffpl, #darkbluekiss, #tomorrow_x_together, #monstax, #kartarpurcorridor, #nctzenselcaday, #lulalivre, #x1flyhightothailand, #goloud, #exodeux, #ongseongwu, #jungkook, #magachallenge, #sidharthshukla, #thankyougamefreak, #jimin, #superm, #biggboss13, #fcbbvb, #peckpalitchoke, #happyleoday, #sozlesmelisehadet, #nowplaying, #1millonsmdqmq, #bambam, #jaehyun, #leeeunsang, #kimyohan, #bb13, #follow, #seventeen, #mark, #baekhyun, #sehun, #nintendoswitch, #amakhosi4life, #leiars, #punbnk48, #deathstranding, #chile, #singapore, #fgo_ep7, #kristperawat, #dutchieyoghurt, #porn, #namdohyon, #gmmtv, #sao_anime, #10kasim, #rmliga, #dearpresidenttrump, #salmankhan, #sb29psg, #bnk48, #soobin, #hanseungwoo"
hashtags = [tags.strip() for tags in hashtags.split(',')]

In [84]:
for hasht in hashtags:
    tweets = ret.getTweets(hasht, 1)
    print("***********************",hasht,"*******************************")
    for tweet in tweets:
        print(tweet.text)
        print("=====================")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

*********************** #x1 *******************************
pt.1 no matter how/what the line up gonna be THEY WONT GET DISBANDED OR NONE OF THE MEMBER GONNA BE KICK OUT OF THE GROUP. THATS IT , ONCE THEYRE ONE THEY ALWAYS ONE  #X1 #X1YoureLoved #StandUpWithX1 #StandUpForX1 #StandUpX1 #엑스원 #엑스원_사랑해 #OneIt #OneItforX1 #oneitlovex1
he’s literally oozing lead drama actor vibes 
© mnm
#강민희 #KANGMINHEE #엑스원 #X1 https://t.co/Wkd1MOdZl4
[#X1] จังหวัดที่ถ่ายทำ 🇹🇭(((comfirmed))) 🌷💘✨
〰️ 30.- 
contact : dm

[Plan X1] 300.-
ㅡ Location
ㅡ Hotel
ㅡ date &amp; time
ㅡ another details

Free update realtime
(dm)

#ตลาดนัดX1 
#KPOPFESTAinBKK
#XONE 
#ตลาดนัดXONE 
#ตลาดนัดONEIT
#X1_FLAϟH

 https://t.co/ETswkIJlzH
Look at their droopy shoulders and thinner frames. Look at their distraught faces. 😔

There's a limit as to how much stress one can take. 

It's taking a toll on their physical and emotional well being.

Stop punishing the boys for what clearly is others' wrongdoing.

#X1 e https://t.co/iKDhhmwI4w
Ha

*********************** #got7 *******************************
👩🏻 : @GOT7Official  never dissapoints me 🥺👌🏻💚

#GOT7 #GOT7_YouCallingMyName  #mamavote #GOT7_CallMyName  #IGOT7 https://t.co/8sTjxIweKz
Who's your bias from #got7 ?💜
 https://t.co/Fiigrnl2od
@Rose2Blossom90 @JacksonWang852 The only fashionista with a clothes line but models topless 
#MIRRORS
#GOT7 
@GOT7Official 
#GOT7WORLDTOUR
#GOT7_KEEPSPINNING
#GOT7_SPINNINGTOP
#GOT7_니가부르는나의이름
#GOT7_CallMyName
#GOT7_YouCallingMyName
can I curse?
#GOT7 #JB #제이비 https://t.co/9eK3wtT6YC
MY GOT7 ALBUM SHIPPED HELL YEAH #GOT7 #GOT7_YouCallingMyName #GOT7_CallMyName
Anyone like the B sides of GOT7s Call My Name album more than the title? Like I enjoy the title track don't get me wrong but its not my favourite so can't see myself playing it a lot. I prefer Crash &amp; Burn or Pray as titles more. Overall great album though ❤
#GOT7
#AGHASE
🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨 ▶️ MCD: https://t.co/rCbrTTAueI
MV: https://t.co/Qyf7LBL0aB ◀️#갓세븐    
#GOT7    
#IGOT7    
#GOT7

*********************** #bts *******************************
@dreamer_49_ @BTSVotingFam @BTS_twt If you’re using your phone to vote, switching between data...WiFi...data... works as well.

You can also reset your WiFi.

@BTS_twt #bts #POPTimeAwards #GrupoDoAno
I’ll never forget the day I discovered how amazing BTS is.  I’ve found a family with my #BTSARMY friends and I’m so proud of the accomplishments #BTS and #ARMY have made.  Let’s continue to make history!  #ARMYAppreciationDay 💜 https://t.co/VybCXhVO2q
@Fratae_tae @NoLoveWithout_V @BTS_twt No no no, we absolutely vote💜
#MAMAVOTE #BTS @BTS_twt
I voted for BTS for Favorite Duo or Group – Pop/Rock at the @AMAs! Vote here: https://t.co/W5FeuhJMgI
#MAMAVOTE #BTS @BTS_twt
@amandarachlee aww she saw my tweet i feel hugged 🥺💜

(#MAMAVOTE #bts @BTS_twt)
@jeonietaehie @BTS_twt Yes, yes I have. #MAMAVOTE #BTS @BTS_twt
@allkpop #JIMIN 💛 we miss you 💕💕              #JIMIN 
#jiminlovely 
#Jiminyouareperfect 
#BTS @BTS_twt https://t.co/eCronKA9o

*********************** #uwmaep1 *******************************
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
*********************** #happy_dohyon_day *******************************
Selamat ulang tahun bayik gede nya aku, kesayangan nya aku🎉 stay healthy, and stay confident that your career will be better in the future.✨ i love youuuu  @x1members ❤️❤️

#JUST_DOHYON_DAY #HAPPY_DOHYON_DAY 
#찬란한열여섯남도현_생일축하해 https://t.co/TJRbdLneWe
It’s 4:25 AM KST and
 #찬란한열여섯남도현_생일축하해 is trending #1 in Seoul Korea. @x1official101  @x1members #JUST_DOHYON_DAY #HAPPY_DOHYON_DAY Can you feel the love all the way to Bangkok  Thailand. https://t.co/ViAaJymxWF
Happy birthday to my son dodo🎂💖
Thank you for being born with a cuteness that exceeds the limit . Thank you for being a part of X1. I love ya~
🍔🌭🍟🍕🍛🍣🥟🍤🍙🍚🍭🍫🍬
.
.
@x1members @x1official101 
#JUST_DOHYON_DAY #HappyBirthdayDohyon #HAPPY_DOHYON_DAY https://t.co/fPhJS25DEQ
hb to the babiest of the babie

*********************** #monsta_x *******************************
DON'T ENGAGE with those accounts, just report asap and send screenshots to Starship. 
They try to drag down Jooheon, we can let them, so please MBB work quick and send only to people you trust. 

#monstax #monsta_x
Coming up in 1 hour on KPOP Listeners’ Choice Top 30 Countdown! Tune in to see if your favorite songs made it this week! 
#TWICE #VIXX #Ateez #Dreamcatcher #CLC #Onewe #SuperJunior #Hinapia  #GOT7 #Victon #Straykids #VAV #Taeyeon #Monsta_X #Day6 and more!
https://t.co/DceAKWvBvN
They start as 7 they better end as 7 #MONSTA_X #MonbebeUnitedForChange https://t.co/Ec03gJB4OY
[FIGHT FOR WONHO: WE ARE HERE FOR #MONSTA_X - A Monsta X Philippines' Support Ad for MONSTA X by #MONBEBE

WONHO, DID YOU EAT WELL?
MX = 7

#UROurWorldMONSTAX
#StarshipProtectMonstaX7
#멈추지_않길_나_약속할게 
@OfficialMonstaX https://t.co/jMVlkcEOzV
Gangnam Hottrack Pansa❤️🐻OMG he’s always so handsome no matter what. 

©️to the respective owners

#sho

...

*********************** #mark *******************************
Former Jets Player Mark Gastineau Says He Was Raped as a Kid https://t.co/msfEpkPJwe - #Gastineau #Jets #Kid #Mark #Player #Raped https://t.co/J0EZv989O7
🎀𝒴𝑜𝓊 𝓈𝒽𝒾𝓃𝑒 𝓁𝒾𝓀𝑒 𝓉𝒽𝑒 𝓈𝓉𝒶𝓇𝓈✨

 #NCTzenSelcaDay #MARK https://t.co/nD9eIM4wgR
Listen Live 📻 ☞ https://t.co/sgNOy2s84U #AiiR #NowPlaying:
♪♫ #HappyBirthday #Mark ♪♫
NP Evil Creek ~ Everything's Gone Gray / For The Last Time (Bonus Track) https://t.co/L3iHCr4tML https://t.co/YN30iXh0B5 Ottawa, Ontario @evil_creek
Hope you're having an great one! 😎 🤘 https://t.co/vahC2CC1oE
#CallMyName album photo shoot 
#MARK MY PRINCE IN WHITE 🥰🥰💚💚❤️❤️

HD Photos 
https://t.co/tqNKhEu84I
https://t.co/csbEDbD6Me
https://t.co/2pQvT3NV0j
https://t.co/qSM06P3Djb
https://t.co/j4ka86vRKb

#MARKTUAN #段宜恩 
#GOT7 #GOT7_CALLMYNAME #GOT7_YOUCALLINGMYNAME https://t.co/OxrYODPx47
-𝘵𝘸𝘰 𝘴𝘰𝘶𝘭𝘴 𝘥𝘰𝘯𝘵 𝘧𝘪𝘯𝘥 𝘦𝘢𝘤𝘩 𝘰𝘵𝘩𝘦𝘳 𝘣𝘺 𝘴𝘪𝘮𝘱𝘭𝘦 𝘢𝘤𝘤𝘪𝘥𝘦𝘯𝘵.-

#NCTzenSelcaDay #MARK https://t.co/frr3cl7HJK
Stan Choi Youngjae,
Stan Tale

*********************** #nintendoswitch *******************************
@Nastadon you motherfu....., hahaha. #SuperMarioMaker2 #NintendoSwitch https://t.co/jRs3g6iHs9
^_T FINALLY!!!!!! #StreamHighlights #Splatoon2 #NintendoSwitch https://t.co/cQgjisCvva
Didn't even know he was exactly there #Splatoon2 #NintendoSwitch https://t.co/zWjPWWBXLn
#NintendoSwitch LORD HELP ME FOR I AM ABOUT TO STAN THIS DISNEY PRINCES https://t.co/9PEhTKctyx
Just did pre order the the Sword version yesterday. Seven more days left for this game releases. #PokemonSword&amp;Shield | #NintendoSwitch https://t.co/9shvFDZB0V
Got 3 wins today to now 6 squads wins #NintendoSwitch https://t.co/wxwPrH4Q5g
They dancin' #SuperMarioMaker2 #NintendoSwitch https://t.co/T0p4rc03q0
speedrun #Bayonetta #NintendoSwitch https://t.co/jHxGiBhIk1
FIRST EVER 30 KO AVERAGE??? #Tetris99 #Tetris #NintendoSwitch https://t.co/fkXbarDWKK
#SmashBros #SmashBrosUltimate #NintendoSwitch TIL that Terry Bogard's final smash breaks G&amp;W's 2d 

*********************** #chile *******************************
For you World Travelers and #wanderlusters: how to travel through #Chile on a budget https://t.co/lvQTeIXgfc
Mon Laferte en Berlín - Tu falta de querer,  FUNERAL,  Mi buen amor - HE... https://t.co/ldDOSgDSyY via @YouTube 

QUEEN OF LATIN POP in Germany
She cancelled tour to help her people in #Chile
Toured #CHILE ~ FREE CONCERTS
FAIR: Media Conceal #Chile’s State Criminality, Delegitimize #Bolivia Democracy. This nakedly biased #journalism, demonizing #Washington’s foes &amp; covering for its clients, is very much par for the course, yet many people still can't see it. #CorporateMedia https://t.co/j7duXgtzG0
#Chile October 18th 2019. These words came to live here NOW !!! https://t.co/SQ1YhZKTzz
#Chile live #ChileDesperto
https://t.co/hVJBJAeJPB
🇨🇱

‘#Chile Woke Up’: #Dictatorship’s Legacy of Inequality Triggers Mass #ChileProtests

@bbcmundo @elmundoes @el_pais @nytimes @CNBC @guardian @TheSun @washingtonpost @lemondefr #B

*********************** #gmmtv *******************************
I just can’t even imagine Tay ever flipping someone off!! Haha his nature is so different from Pete it seems! It’s what makes watching him be Pete so entertaining! He does it so well!! @Tawan_V #DarkBlueKissEP5 #DarkBlueKiss #GMMTV https://t.co/giyLXkQYS7
Next station &gt;&gt;&gt;&gt;&gt;
#TayNewFMinChengdu ❤
#Newwiee #GMMTV 
Buy ticket :: https://t.co/BFFNlcifGP https://t.co/oqXu0PqKD3
Well I think at this time, P’Sun is almost confirmed Mork’s feeling and he START to...YOU KNOW😈this is my fav scene so far💙but I pretty sure that I love the next episode more😈
#DarkBlueKissEP5
#DarkBlueKiss 
#GMMTV 
#GawinCaskeyy
@plapodd https://t.co/BIIW7BEyhJ
Feeling shy? P’Sun’s reaction is cute🥰 (still pretending I know Thai language😭)
@plapodd 
#DarkBlueKissEP5 
#DarkBlueKiss 
#GMMTV https://t.co/Et38jm7mM8
P'Sun is so handsome😫😫😫
can't wait the next episode💙
@plapodd 
#DarkBlueKissEP5 
#DarkBlueKiss 
#GMMTV 
#GawinCaskeyy https://t.co

*********************** #soobin *******************************
I just love how soobin stopped using filters cuz Maaa his skin is glowing and we love to see that 😌❣
#TOMORROW_X_TOGETHER @TXT_bighit @TXT_members #SOOBIN https://t.co/GpH0T1OAsJ
Where are you MOAs?  Please vote for soobin  on star play! #TOMORROW_X_TOGETHER #TXT #SOOBIN #TomorrowXTogether https://t.co/y1OkeiTngq
I found this on Instagram. OMG he so sexy!!! #TXT #SOOBIN #CHOISOOBIN #TomorrowXTogether https://t.co/m7VZz2baqt
pre-debut 

#수빈 x #범규
#SOOBIN x #BEOMGYU 
@TXT_members @TXT_bighit https://t.co/OaLMVbckte
191109  #soobin #투모로우바이투게더 #TOMORROW_X_TOGETHER #TXT 
@TXT_members

©~JellyBean_bin
 https://t.co/PIagInfjVR
191109  #soobin #투모로우바이투게더 #TOMORROW_X_TOGETHER #TXT 
@TXT_members

©~JellyBean_bin

 https://t.co/3tTxKZS8vp
Their facial expressions... 😍
Love the choreo🔥
(Part 2)
#SOOBIN #YEONJUN #BEOMGYU #TAEHYUN #HUENINGKAI @TXT_members https://t.co/uemjBjcRW3
Cuuuuuteeee~! 😆🐰🐱

#MAMAVOTE #txt @TXT_members 
#SOOBIN #Y